In [25]:
import dash
import dash_core_components as dcc
import dash_html_components as html

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

colors = {
    'background': '#111111',
    'text': '#7FDBFF'
}

app.layout = html.Div(style={'backgroundColor': colors['background']}, children=[
    html.H1(
        children='Hello Dash',
        style={
            'textAlign': 'center',
            'color': colors['text']
        }
    ),

    html.Div(children='Dash: A web application framework for Python.', style={
        'textAlign': 'center',
        'color': colors['text']
    }),

    dcc.Graph(
        id='example-graph-2',
        figure={
            'data': [
                {'x': [1, 2, 3], 'y': [4, 1, 2], 'type': 'bar', 'name': 'SF'},
                {'x': [1, 2, 3], 'y': [2, 4, 5], 'type': 'bar', 'name': u'Montréal'},
            ],
            'layout': {
                'plot_bgcolor': colors['background'],
                'paper_bgcolor': colors['background'],
                'font': {
                    'color': colors['text']
                }
            }
        }
    )
])

if __name__ == '__main__':
    app.run_server(debug=True)

Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Debugger PIN: 098-040-089
Debugger PIN: 098-040-089


SystemExit: 1

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2918: UserWarning:

To exit: use 'exit', 'quit', or Ctrl-D.



# Como el trabajo con DASH no funciono se usara plotly para los graficos interactivos

In [15]:
import plotly.graph_objs as go
fig= go.FigureWidget(
    data=[
        dict(
            type='scattergl',
            x=[1,2,3,4,5],
            y=[3,6,8,4,7],
            mode='markers',
        )
    ],
)
fig.layout.title="Ejemplo"
fig.layout.xaxis.title="X"
fig.layout.yaxis.title="Y"
fig2=fig
test=fig.add_scatter(x=[1,2,3,4,5],y=[9,8,9,8,9])
from ipywidgets import HBox,VBox
VBox([fig,fig2,HBox([fig2,fig])])

VBox(children=(FigureWidget({
    'data': [{'mode': 'markers',
              'type': 'scattergl',
              'uid': 'aadba9f8-d3e1-4f69-b310-2e333d073585',
              'x': [1, 2, 3, 4, 5],
              'y': [3, 6, 8, 4, 7]},
             {'type': 'scatter',
              'uid': 'b433946f-2a27-4e92-848f-19175cbbcf6f',
              'x': [1, 2, 3, 4, 5],
              'y': [9, 8, 9, 8, 9]}],
    'layout': {'title': {'text': 'Ejemplo'}, 'xaxis': {'title': {'text': 'X'}}, 'yaxis': {'title': {'text': 'Y'}}}
}), FigureWidget({
    'data': [{'mode': 'markers',
              'type': 'scattergl',
              'uid': 'aadba9f8-d3e1-4f69-b310-2e333d073585',
              'x': [1, 2, 3, 4, 5],
              'y': [3, 6, 8, 4, 7]},
             {'type': 'scatter',
              'uid': 'b433946f-2a27-4e92-848f-19175cbbcf6f',
              'x': [1, 2, 3, 4, 5],
              'y': [9, 8, 9, 8, 9]}],
    'layout': {'title': {'text': 'Ejemplo'}, 'xaxis': {'title': {'text': 'X'}}, 'yaxis': {'title': {'text': 'Y'}}}
}), HBox(children=(FigureWidget({
    'data': [{'mode': 'markers',
              'type': 'scattergl',
              'uid': 'aadba9f8-d3e1-4f69-b310-2e333d073585',
              'x': [1, 2, 3, 4, 5],
              'y': [3, 6, 8, 4, 7]},
             {'type': 'scatter',
              'uid': 'b433946f-2a27-4e92-848f-19175cbbcf6f',
              'x': [1, 2, 3, 4, 5],
              'y': [9, 8, 9, 8, 9]}],
    'layout': {'title': {'text': 'Ejemplo'}, 'xaxis': {'title': {'text': 'X'}}, 'yaxis': {'title': {'text': 'Y'}}}
}), FigureWidget({
    'data': [{'mode': 'markers',
              'type': 'scattergl',
              'uid': 'aadba9f8-d3e1-4f69-b310-2e333d073585',
              'x': [1, 2, 3, 4, 5],
              'y': [3, 6, 8, 4, 7]},
             {'type': 'scatter',
              'uid': 'b433946f-2a27-4e92-848f-19175cbbcf6f',
              'x': [1, 2, 3, 4, 5],
              'y': [9, 8, 9, 8, 9]}],
    'layout': {'title': {'text': 'Ejemplo'}, 'xaxis': {'title': {'text': 'X'}}, 'yaxis': {'title': {'text': 'Y'}}}
})))))

In [16]:
from plotly.offline import iplot, init_notebook_mode
import plotly.graph_objs as go
import plotly.io as pio

import os
import numpy as np
init_notebook_mode(connected=True)
N = 100
x = np.random.rand(N)
y = np.random.rand(N)
colors = np.random.rand(N)
sz = np.random.rand(N)*30

fig = go.Figure()
fig.add_scatter(x=x,
                y=y,
                mode='markers',
                marker={'size': sz,
                        'color': colors,
                        'opacity': 0.6,
                        'colorscale': 'Viridis'
                       });
iplot(fig)

In [17]:
import plotly.offline as off

off.init_notebook_mode(connected=True)

subject = ['Moe','Larry','Curly','Moe','Larry','Curly','Moe','Larry','Curly','Moe','Larry','Curly']
score = [1,6,2,8,2,9,4,5,1,5,2,8]

data = [dict(
  type = 'scatter',
  x = subject,
  y = score,
  mode = 'markers',
  transforms = [dict(
    type = 'groupby',
    groups = subject,
    styles = [
        dict(target = 'Moe', value = dict(marker = dict(color = 'blue'))),
        dict(target = 'Larry', value = dict(marker = dict(color = 'red'))),
        dict(target = 'Curly', value = dict(marker = dict(color = 'black')))
    ]
  )]
)]

off.iplot({'data': data}, validate=False)

# Para leer los datos directos del sistema se utilizaran las librerias para manejar archivos XML

In [18]:
import pandas as pd
import numpy as np
import xml.etree.ElementTree
import datetime
import matplotlib.pyplot as plt
import distutils
from distutils import util
import plotly.graph_objs as go

def process_data(x):
    e = xml.etree.ElementTree.parse(x).getroot()
    df=pd.DataFrame(list(map(lambda y:list(map(lambda x: x.text,y)),e.getchildren())))
    cols=list(map(lambda x:x.tag,e.getchildren()[1].getchildren()))
    df.columns=cols
    df.index=df.iloc[:,0]
    df=np.transpose(np.transpose(df).loc[~pd.isnull(np.transpose(df.iloc[1,:]))]).iloc[1:len(df),:]
    return df

In [13]:
df=process_data('cheques.xml')
df

,folio,numcheque,fecha,cierre,mesa,nopersonas,idmesero,pagado,cancelado,impreso,...,c_iddispositivo,esalestatus,statussr,deliverycharge,comandaimpresa,foodorder,cashpaymentwith,intentoenvioaf,paymentmethod_id,tkc_recompensa
folio,,,,,,,,,,,,,,,,,,,,,
9496,9496,9031,2019-02-01T09:22:40,2019-02-01T09:27:22,LLVR,1,08,true,false,true,...,-1,-1,-1,0.00000,false,-1,-1.0000,0,-1,0.0000
9497,9497,9049,2019-02-01T10:59:34,2019-02-01T15:29:34,MAURICO-51B65,1,08,true,false,true,...,-1,-1,-1,0.00000,false,-1,-1.0000,0,-1,0.0000
9498,9498,9032,2019-02-01T11:51:08,2019-02-01T12:03:14,6,1,08,true,false,true,...,-1,-1,-1,0.00000,false,-1,-1.0000,0,-1,0.0000
9499,9499,9048,2019-02-01T12:09:34,2019-02-01T15:29:23,BRUNO-AE289,1,08,true,false,true,...,-1,-1,-1,0.00000,false,-1,-1.0000,0,-1,0.0000
9500,9500,9033,2019-02-01T12:17:24,2019-02-01T12:47:21,7,9,08,true,false,true,...,-1,-1,-1,0.00000,false,-1,-1.0000,0,-1,0.0000
9501,9501,9034,2019-02-01T12:52:24,2019-02-01T13:04:15,LLEVAR,1,08,true,false,true,...,-1,-1,-1,0.00000,false,-1,-1.0000,0,-1,0.0000
9502,9502,9035,2019-02-01T12:52:48,2019-02-01T13:14:56,5,9,08,true,false,true,...,-1,-1,-1,0.00000,false,-1,-1.0000,0,-1,0.0000
9503,9503,9047,2019-02-01T12:55:47,2019-02-01T15:29:15,90AAB-JAVIER,1,08,true,false,true,...,-1,-1,-1,0.00000,false,-1,-1.0000,0,-1,0.0000
9504,9504,9036,2019-02-01T12:57:12,2019-02-01T13:27:06,9,9,08,true,false,true,...,-1,-1,-1,0.00000,false,-1,-1.0000,0,-1,0.0000


### Convertir datos extraidos del archivo cheques en datos manejables

In [14]:
df.iloc[:,2]=list(map(lambda x:np.datetime64(x),df.iloc[:,2]))
df.iloc[:,3]=list(map(lambda x:np.datetime64(x),df.iloc[:,3]))
#x.astype('M8[ms]').astype('O').hour
df['total']=list(map(lambda x:float(x),df['total']))
df['totalalimentos']=list(map(lambda x:float(x),df['totalalimentos']))
df['totalbebidas']=list(map(lambda x:float(x),df['totalbebidas']))
df['totalotros']=list(map(lambda x:float(x),df['totalotros']))
df['totaldescuentoalimentos']=list(map(lambda x:float(x),df['totaldescuentoalimentos']))
df['totaldescuentobebidas']=list(map(lambda x:float(x),df['totaldescuentobebidas']))
df['totaldescuentobebidas']=list(map(lambda x:float(x),df['totaldescuentobebidas']))
df['efectivo']=list(map(lambda x:float(x),df['efectivo']))
df['tarjeta']=list(map(lambda x:float(x),df['tarjeta']))
df['nopersonas']=list(map(lambda x:int(x),df['nopersonas']))
df['cancelado']=list(map(lambda x:distutils.util.strtobool(x),df['cancelado']))
df['idempresa']=list(map(lambda x:int(x),df['idempresa']))

#df.columns.tolist()

In [77]:
process_data('cheqdet.xml')

,foliodet,movimiento,cantidad,idproducto,descuento,precio,impuesto1,impuesto2,impuesto3,preciosinimpuestos,...,sistema_envio,idturno_cierre,procesado,promovolumen,iddispositivo,productsyncidsr,subtotalsrx,totalsrx,idmovtobillar,tipocambio
foliodet,,,,,,,,,,,,,,,,,,,,,
9496,9496,1,1.000000,TPTCL02,0.00,52.0000,16.00,0.00,0.00,44.8276,...,1,468,true,false,-1,-1,-1.00000,-1.00000,0,0.0000
9496,9496,2,1.000000,TPTCL05,0.00,52.0000,16.00,0.00,0.00,44.8276,...,1,468,true,false,-1,-1,-1.00000,-1.00000,0,0.0000
9496,9496,3,2.000000,LTDDF02,0.00,26.0000,16.00,0.00,0.00,22.4138,...,1,468,true,false,-1,-1,-1.00000,-1.00000,0,0.0000
9496,9496,4,1.000000,LTDDF01,0.00,12.0000,16.00,0.00,0.00,10.3448,...,1,468,true,false,-1,-1,-1.00000,-1.00000,0,0.0000
9497,9497,1,1.000000,BEJUG02,0.00,22.0000,16.00,0.00,0.00,18.9655,...,1,468,true,false,-1,-1,-1.00000,-1.00000,0,0.0000
9497,9497,2,1.000000,LTGOB01,0.00,70.0000,16.00,0.00,0.00,60.3448,...,1,468,true,false,-1,-1,-1.00000,-1.00000,0,0.0000
9497,9497,3,3.000000,LTDDP04,0.00,26.0000,16.00,0.00,0.00,22.4138,...,1,468,true,false,-1,-1,-1.00000,-1.00000,0,0.0000
9498,9498,1,1.000000,BEAGU09,0.00,17.0000,16.00,0.00,0.00,14.6552,...,1,468,true,false,-1,-1,-1.00000,-1.00000,0,0.0000
9498,9498,2,1.000000,LTDDP05,0.00,26.0000,16.00,0.00,0.00,22.4138,...,1,468,true,false,-1,-1,-1.00000,-1.00000,0,0.0000


In [110]:
from plotly.offline import iplot, init_notebook_mode
import plotly.graph_objs as go
import plotly.io as pio

import os
import numpy as np
init_notebook_mode(connected=True)

class plotting:
    def __init__(self):
        self.fig=go.Figure()
    def pie(self,df):
        self.fig.add_pie=self.fig.add_pie(labels=list(map(lambda x:x.weekday(),df.fecha)),
            values=df.total,);
        iplot(self.fig)
        
pie_chart=plotting()
pie_chart.pie(df)